In [1]:
pip install -r requeriments.txt

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\virtual_env\Sonia\SoniaM\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import sqlite3 as sql
import numpy as np
import pandas as pd

import seaborn as sns
import plotly.express as px 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist

from yellowbrick.target import FeatureCorrelation

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict
import difflib

import json

import warnings
warnings.filterwarnings("ignore")

In [3]:
con = sql.connect("spotify.sqlite")
con.text_factory = lambda b: b.decode(errors = 'ignore')
df = pd.read_sql_query("SELECT * from tracks", con)
df_genres = pd.read_sql_query("SELECT * from genres", con)
df_a_features = pd.read_sql_query("SELECT * from audio_features", con)
con.close()

In [4]:
df.describe()

,disc_number,duration,explicit,track_number,popularity,is_playable
count,8.741672e+06,8.741672e+06,8.741672e+06,8.741672e+06,8.741672e+06,413800.000000
mean,1.137497e+00,2.369687e+05,6.679889e-02,1.093178e+01,6.118262e+00,0.875391
std,9.958747e-01,1.596887e+05,2.496734e-01,2.272236e+01,1.057977e+01,0.330275
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000
25%,1.000000e+00,1.692000e+05,0.000000e+00,4.000000e+00,0.000000e+00,1.000000
50%,1.000000e+00,2.161600e+05,0.000000e+00,7.000000e+00,1.000000e+00,1.000000
75%,1.000000e+00,2.738870e+05,0.000000e+00,1.200000e+01,8.000000e+00,1.000000
max,6.300000e+01,1.967206e+07,1.000000e+00,1.522000e+03,1.000000e+02,1.000000


In [5]:
df.head()

,id,disc_number,duration,explicit,audio_feature_id,name,preview_url,track_number,popularity,is_playable
0,1dizvxctg9dHEyaYTFufVi,1,275893,1,1dizvxctg9dHEyaYTFufVi,Gz And Hustlas (feat. Nancy Fletcher),,12,0,NaN
1,2g8HN35AnVGIk7B8yMucww,1,252746,1,2g8HN35AnVGIk7B8yMucww,Big Poppa - 2005 Remaster,https://p.scdn.co/mp3-preview/770e023eb0318270...,13,77,NaN
2,49pnyECzcMGCKAqxfTB4JZ,3,315080,0,49pnyECzcMGCKAqxfTB4JZ,"You Were Born - Early Version Of ""One Of The T...",,6,8,1.0
3,4E5IFAXCob6QqZaJMTw5YN,1,240800,1,4E5IFAXCob6QqZaJMTw5YN,Poppin' Them Thangs,https://p.scdn.co/mp3-preview/f3b556ced9657f89...,2,70,NaN
4,1gSt2UlC7mtRtJIc5zqKWn,2,203666,0,1gSt2UlC7mtRtJIc5zqKWn,"It's Hard To Say ""I Do"", When I Don't",,2,50,NaN


In [6]:
df_a_features.describe()

,acousticness,danceability,duration,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
count,8.740043e+06,8.740043e+06,8.740043e+06,8.740043e+06,8.740043e+06,8.740043e+06,8.740043e+06,8.740043e+06,8.740043e+06,8.740043e+06,8.740043e+06,8.740043e+06,8.740043e+06
mean,4.190219e-01,5.330364e-01,2.368114e+05,5.475726e-01,2.557578e-01,5.241622e+00,2.065516e-01,-1.079117e+01,6.603950e-01,9.627075e-02,1.186613e+02,3.840387e+00,4.803654e-01
std,3.756609e-01,1.916363e-01,1.546432e+05,2.817997e-01,3.683377e-01,3.542915e+00,1.818429e-01,6.280323e+00,4.735752e-01,1.407174e-01,3.104153e+01,5.667822e-01,2.800947e-01
min,0.000000e+00,0.000000e+00,1.000000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-6.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.300000e-02,4.000000e-01,1.692270e+05,3.170000e-01,1.110000e-06,2.000000e+00,9.630000e-02,-1.352100e+01,0.000000e+00,3.590000e-02,9.522100e+01,4.000000e+00,2.390000e-01
50%,3.310000e-01,5.490000e-01,2.161600e+05,5.710000e-01,1.780000e-03,5.000000e+00,1.300000e-01,-9.136000e+00,1.000000e+00,4.730000e-02,1.190980e+02,4.000000e+00,4.750000e-01
75%,8.110000e-01,6.790000e-01,2.738930e+05,7.910000e-01,6.330000e-01,8.000000e+00,2.640000e-01,-6.364000e+00,1.000000e+00,8.330000e-02,1.375130e+02,4.000000e+00,7.150000e-01
max,9.960000e-01,1.000000e+00,6.072187e+06,1.000000e+00,1.000000e+00,1.100000e+01,1.000000e+00,6.275000e+00,1.000000e+00,9.740000e-01,2.499870e+02,5.000000e+00,1.000000e+00


In [7]:
df_a_features.head()

,id,acousticness,analysis_url,danceability,duration,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,2jKoVlU7VAmExKJ1Jh3w9P,0.1800,https://api.spotify.com/v1/audio-analysis/2jKo...,0.893,219160,0.514,0.000000,11,0.0596,-5.080,1,0.283,95.848000,4,0.787
1,4JYUDRtPZuVNi7FAnbHyux,0.2720,https://api.spotify.com/v1/audio-analysis/4JYU...,0.520,302013,0.847,0.000000,9,0.3250,-5.300,1,0.427,177.371002,4,0.799
2,6YjKAkDYmlasMqYw73iB0w,0.0783,https://api.spotify.com/v1/audio-analysis/6YjK...,0.918,288200,0.586,0.000000,1,0.1450,-2.890,1,0.133,95.516998,4,0.779
3,2YlvHjDb4Tyxl4A1IcDhAe,0.5840,https://api.spotify.com/v1/audio-analysis/2Ylv...,0.877,243013,0.681,0.000000,1,0.1190,-6.277,0,0.259,94.834999,4,0.839
4,3UOuBNEin5peSRqdzvlnWM,0.1700,https://api.spotify.com/v1/audio-analysis/3UOu...,0.814,270667,0.781,0.000518,11,0.0520,-3.330,1,0.233,93.445000,4,0.536


In [8]:
df_genres.describe()

,id
count,5489
unique,5489
top,detroit hip hop
freq,1


In [9]:
df_genres.head()

,id
0,detroit hip hop
1,g funk
2,gangster rap
3,hardcore hip hop
4,hip hop


In [10]:
df_t_f=df
features_n = ['valence','acousticness', 'danceability', 'energy', 'instrumentalness',
              'liveness', 'loudness', 'speechiness', 'tempo','duration','key','mode']
length = len(features_n)
for i in range(length):
    df_t_f[features_n[i]] = df_t_f.id.map(df_a_features.set_index('id')[features_n[i]])

In [12]:
df_t_f.loc[df['id'] == '1dizvxctg9dHEyaYTFufVi']

,id,disc_number,duration,explicit,audio_feature_id,name,preview_url,track_number,popularity,is_playable,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,key,mode
0,1dizvxctg9dHEyaYTFufVi,1,275893.0,1,1dizvxctg9dHEyaYTFufVi,Gz And Hustlas (feat. Nancy Fletcher),,12,0,NaN,...,0.164,0.652,0.814,0.0,0.36,-4.901,0.31,91.888,1.0,1.0


In [14]:
df_t_f.head()

,id,disc_number,duration,explicit,audio_feature_id,name,preview_url,track_number,popularity,is_playable,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,key,mode
0,1dizvxctg9dHEyaYTFufVi,1,275893.0,1,1dizvxctg9dHEyaYTFufVi,Gz And Hustlas (feat. Nancy Fletcher),,12,0,NaN,...,0.1640,0.652,0.814,0.000,0.3600,-4.901,0.3100,91.888000,1.0,1.0
1,2g8HN35AnVGIk7B8yMucww,1,252747.0,1,2g8HN35AnVGIk7B8yMucww,Big Poppa - 2005 Remaster,https://p.scdn.co/mp3-preview/770e023eb0318270...,13,77,NaN,...,0.4300,0.780,0.575,0.000,0.1430,-7.247,0.2730,84.491997,9.0,0.0
2,49pnyECzcMGCKAqxfTB4JZ,3,315080.0,0,49pnyECzcMGCKAqxfTB4JZ,"You Were Born - Early Version Of ""One Of The T...",,6,8,1.0,...,0.3310,0.624,0.564,0.293,0.2560,-8.928,0.0297,111.960999,7.0,1.0
3,4E5IFAXCob6QqZaJMTw5YN,1,240800.0,1,4E5IFAXCob6QqZaJMTw5YN,Poppin' Them Thangs,https://p.scdn.co/mp3-preview/f3b556ced9657f89...,2,70,NaN,...,0.2630,0.769,0.818,0.000,0.0787,-1.385,0.1920,88.016998,6.0,0.0
4,1gSt2UlC7mtRtJIc5zqKWn,2,203667.0,0,1gSt2UlC7mtRtJIc5zqKWn,"It's Hard To Say ""I Do"", When I Don't",,2,50,NaN,...,0.0196,0.420,0.949,0.000,0.2840,-3.902,0.0823,163.945007,10.0,1.0


In [16]:
df_t_f.replace([np.inf, -np.inf], np.nan, inplace=True)

In [18]:
df_t_f.isnull().sum()

id                        0
disc_number               0
duration               1629
explicit                  0
audio_feature_id          0
name                      0
preview_url               0
track_number              0
popularity                0
is_playable         8327872
valence                1629
acousticness           1629
danceability           1629
energy                 1629
instrumentalness       1629
liveness               1629
loudness               1629
speechiness            1629
tempo                  1629
key                    1629
mode                   1629
dtype: int64

In [20]:
df_t_f.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8741672 entries, 0 to 8741671
Data columns (total 21 columns):
 #   Column            Dtype  
---  ------            -----  
 0   id                object 
 1   disc_number       int64  
 2   duration          float64
 3   explicit          int64  
 4   audio_feature_id  object 
 5   name              object 
 6   preview_url       object 
 7   track_number      int64  
 8   popularity        int64  
 9   is_playable       float64
 10  valence           float64
 11  acousticness      float64
 12  danceability      float64
 13  energy            float64
 14  instrumentalness  float64
 15  liveness          float64
 16  loudness          float64
 17  speechiness       float64
 18  tempo             float64
 19  key               float64
 20  mode              float64
dtypes: float64(13), int64(4), object(4)
memory usage: 1.4+ GB


In [ ]:
df_c=df_t_f.fillna(0)
df_c.isna().sum()

In [ ]:
feature_names = ['acousticness', 'danceability', 'energy', 'instrumentalness',
       'liveness', 'loudness', 'speechiness', 'tempo', 'valence','duration','explicit','key','mode']

X, y = df_c[feature_names], df_c['popularity']

# Creamos una lista con los nombres de las caracteristicas
features = np.array(feature_names)

# Creamos la instancia
visualizer = FeatureCorrelation(labels=features)

plt.rcParams['figure.figsize']=(20,20)
visualizer.fit(X, y)
visualizer.show()

In [ ]:
df_c.to_csv("spotify_tracks.csv")